<font size=50 color=darkblue>Graph Coloring - Breaking Symmetries</font>

# Problem modelling with DOcplex

## Import necessary modules

- DOcplex will be used to model and solve the Graph-Coloring MILP
- Import module `json` to read example data
- Import module `numpy` to vectorize the calculation of node degrees

In [ ]:
from docplex.mp.model import Model
import json
import numpy as np

## Import the input from example files
### 36 examples are available: `coloring_ex_1`, `coloring_ex_2`, $\dots$ , `coloring_ex_36`
**<font color='red'>Note: For CPLEX Community Edition, only the first 7 examples</font><font> `coloring_ex_1`, $\dots$ , `coloring_ex_7`</font><font color='red'> are applicable.</font>**

In [ ]:
with open('coloring_data/coloring_ex_7', 'r') as fp:
    colr_data = json.load(fp)
    
V = list(range(colr_data['n']))
inc_E = colr_data['inc_E']
E = [(int(u),v) for u in inc_E for v in inc_E[u]]
print(f'In this example:\n\t- Number of nodes: {len(V)}\n\t- Number of links: {len(E)}')

## Create the ILP model for Graph-Coloring problem using DOcplex

In [ ]:
colr_ILP = Model(name='Graph Coloring')

## Solution Symmetry
The Graph-Coloring model as we have constructed so far exhibits a lot of solution symmetries. One such symmetry is illustrated in the diagram below, where different solutions can yield identical objective values (the minimum number of colors is 3).

<img src='img/symm.png' width=1000/>

Usually, these symmetries are undesirable as it would require the solver to explore solutions whose similar versions have already been explored.

For the Graph-Coloring problem, to break the symmetry as illustrated above, one can exploit the color indices and introduce redundant constraints based on these color indices. These constraints are regards as "redundant" because they do not affect the optimal results. Their role is merely to limit the search space so that the solver can avoid symmetries.

This symmetry-breaking technique (among others) is based on the observation that, in order to keep the color indices low, a node can be colored with a color index not larger than its <em>node degree</em>. In an undirected network, the degree of a node (or node degree) is the number of its neighboring nodes. For example, the node degrees of the nodes 0, 1, 2, 3 in the diagram above are 1, 3, 1, 1 respectively. Hence, the color indices of these nodes can be limited to 1, 3, 1, 1 respectively. You can prove this observation as homework.

Let us mathematically denote the node degree of a node $v$ as:
$$\partial_{(v)},\quad v\in\mathcal{V}$$

The following cell prepares a list of node degrees, named `node_degree`, for the entire node set $\mathcal{V}$.

In [ ]:
node_degree = np.zeros_like(V)
np.add.at(node_degree, E, 1)
node_degree = node_degree.tolist()
print(f'{node_degree = }')

## Binarization of integer variables
We revisit the binarization version of the Graph-Coloring model and make adjustments to avoid the symmetries.

<font>__In the following cell, please define the binary variables__</font> $\color{darkblue}x_{(v,c)}$ <font>__with the adjusted constraints on their color index subscripts:__</font>

$$\color{darkblue}x_{(v,c)}\in\{0,1\},\quad\forall v\in \mathcal{V}, c\in \{0,\dots,\partial_{(v)}\}$$

In [ ]:
## YOUR CODE HERE ##


Since out of all available colors, each node takes one color only, the following set of constraints hold:
$${\color{darkblue}\sum_{c\in\{0,\dots,\partial_{(v)}\}}x_{(v,c)} = 1,\quad\forall v\in \mathcal{V}\qquad\qquad (*)}$$
__Please introduce these constraints to the optimization model.__

In [ ]:
## YOUR CODE HERE ##


Two connecting nodes (neighboring countries) $u$ and $v$ must not receive the same color $c$ (at most either of $u$ and $v$ can take color $c$). __Please introduce the following constraints to the optimization model.__
$$\color{darkblue}x_{(u,c)} + x_{(v,c)} \le 1,\quad\forall (u,v)\in \mathcal{E}, c\in \{0,\dots,\text{min}\left(\partial_{(u)},\partial_{(v)}\right)\}$$

In [ ]:
## YOUR CODE HERE ##


As a consequence of constraint set $(*)$, the color taken by a node $v$ is expressed as $$\sum_{c\in\{0,\dots,\partial_{(v)}\}}c\cdot {\color{blue}x_{(v,c)}}$$
Hence, the big-M version of ${\color{blue}x_\textbf{MAX}} - {\color{blue}x_{(v)}}\ge 0$ is rewritten as $$\color{darkblue}x_\textbf{MAX} - \sum_{c\in \{0,\dots,\partial_{(v)}\}}c\cdot x_{(v,c)}\ge 0,\qquad\forall v\in \mathcal{V}$$

__Please define the variable <font color='darkblue'>$x_\textbf{MAX} \in\mathbb{R}$</font> and introduce the above set of constraints.__

In [ ]:
## YOUR CODE HERE ##


__Please define the objective function.__
$$\color{darkblue}\textbf{Minimize}\qquad x_\textbf{MAX}$$

In [ ]:
## YOUR CODE HERE ##


## The overall Graph-Coloring ILP (binarization version with symmetry-breaking constraints)
**Minimize**
### $$\color{blue}x_\textbf{MAX}$$
**Subject to**
### \begin{align*}
\sum_{c\in\{0,\dots,\partial_{(v)}\}}{\color{blue}x_{(v,c)}} &= 1,\qquad&\forall v\in \mathcal{V}\\
{\color{blue}x_{(u,c)}} + {\color{blue}x_{(v,c)}} &\le 1,\qquad&\forall (u,v)\in \mathcal{E}, c\in \{0,\dots,\text{min}\left(\partial_{(u)},\partial_{(v)}\right)\}\\
{\color{blue}x_\textbf{MAX}} - \sum_{c\in \{0,\dots,\partial_{(v)}\}}c\cdot {\color{blue}x_{(v)}}&\ge 0,\qquad&\forall v\in \mathcal{V}\\
{\color{blue}x_{(v,c)}}&\in\{0,1\},\qquad& \forall v\in \mathcal{V}, c\in \{0,\dots,\partial_{(v)}\}\\
{\color{blue}x_\textbf{MAX}}&\in\mathbb{R}
\end{align*}

## Summarize the model

In [ ]:
colr_ILP.print_information()

## Set termination criteria (stop if either criterion is satisfied)
- Solution time is not longer than 5 minutes.
- Relative optimality gap (i.e., the gap between the tightest LP relaxed bound and the best integral solution found so far) is at most 2%.

In [ ]:
colr_ILP.set_time_limit(5*60) # 5 minutes
_ = colr_ILP.parameters.mip.tolerances.mipgap.set(0.02)

## Solve the ILP

In [ ]:
colr_sol = colr_ILP.solve(log_output=True)

## Display the result

In [ ]:
if colr_sol:
    colr_sol.display()